# Exploring Airports in Australia

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from config import geoapify_key

## DataFrame Preparation

In [2]:
# Import cities file as DataFrame
cities_pd = pd.read_csv("../Resources/cities.csv")
cities_pd.head()

,City
0,Sydney
1,Melbourne
2,Perth
3,Cairns
4,Adelaide


In [3]:
# Add columns the airports data we will fetch using the Geoapify API
# Note that we used "" to specify initial entry.
cities_pd["Lat"] = ""
cities_pd["Lon"] = ""
cities_pd["Airport Name"] = ""
cities_pd["IATA Name"] = ""
cities_pd["Airport Address"] = ""
cities_pd["Distance"] = ""
cities_pd["Website"] = ""
cities_pd.head()

,City,Lat,Lon,Airport Name,IATA Name,Airport Address,Distance,Website
0,Sydney,,,,,,,
1,Melbourne,,,,,,,
2,Perth,,,,,,,
3,Cairns,,,,,,,
4,Adelaide,,,,,,,


## Look for Geographical Coordinates for Each City

In [8]:
# Define the API parameters
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

In [ ]:
params['text']='Sydney, Aus... '

In [12]:

# Loop through the cities_pd DataFrame and search coordinates for each city
for index, row in cities_pd.iterrows():

    # Get the city's name & add ", Australia" to the string so geoapify finds the correct city
    city = row["City"] + ", Australia"
    # Add the current city to the parameters
    params['text']=city

    # Make the API request
    response = requests.get(base_url, params=params)
    # Convert response to JSON
    response = response.json()

    # Extract latitude and longitude
    cities_pd.loc[index, "Lat"] = response["results"][0]["lat"]
    cities_pd.loc[index, "Lon"] = response["results"][0]["lon"]

    # Log the search results
#print(f"Coordinates for {city} fetched...")
    
# Display sample data to confirm that the coordinates appear


In [13]:
cities_pd.head(2)

,City,Lat,Lon,Airport Name,IATA Name,Airport Address,Distance,Website
0,Sydney,-33.869844,151.208285,,,,,
1,Melbourne,-37.814245,144.963173,,,,,


In [ ]:
#lat,long

## Retrieve Airports' Information

In [14]:
# Set parameters to search for airport's details
radius = 50000
params = {
    "categories":"airport",
    "apiKey": geoapify_key,
    "limit":20
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Iterate through the types_df DataFrame
for index, row in cities_pd.iterrows():
    # Get the city's name
    city = row["City"]
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lon"]
    
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
           
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
        
    # Convert the API response to JSON format
    response = response.json()
    # Grab the first airport from the results and store the details in the DataFrame
    try:
        cities_pd.loc[index, "Airport Name"] = response["features"][0]["properties"]["name"]
        cities_pd.loc[index, "IATA Name"] = response["features"][0]["properties"]["datasource"]["raw"]["iata"]
        cities_pd.loc[index, "Airport Address"] = response["features"][0]["properties"]["address_line2"]
        cities_pd.loc[index, "Distance"] = response["features"][0]["properties"]["distance"]
        cities_pd.loc[index, "Website"] = response["features"][0]["properties"]["datasource"]["raw"]["website"]

    except KeyError as e:
        # If no airport is found, log the error.
        print(f"{e.args[0]} not found for {cities_pd.loc[index, 'Airport Name']}")
        
    # Log the search results
    print(f"nearest airport from {city}: {cities_pd.loc[index, 'Airport Name']}")

# Display sample data


nearest airport from Sydney: Sydney (Kingsford Smith) Airport
nearest airport from Melbourne: Essendon Fields Airport
nearest airport from Perth: Perth Airport
website not found for Cairns Airport
nearest airport from Cairns: Cairns Airport
nearest airport from Adelaide: Adelaide Airport
nearest airport from Gold Coast: Gold Coast Airport
nearest airport from Townsville: Townsville Airport
nearest airport from Williamtown: Newcastle Airport


In [16]:
# Save Data to CSV
cities_pd.head(2)

,City,Lat,Lon,Airport Name,IATA Name,Airport Address,Distance,Website
0,Sydney,-33.869844,151.208285,Sydney (Kingsford Smith) Airport,SYD,"O'Riordan Street, Mascot NSW 2020, Australia",6742,https://www.sydneyairport.com.au/
1,Melbourne,-37.814245,144.963173,Essendon Fields Airport,MEB,"72 Hargrave Avenue, Essendon Fields VIC 3041, ...",9983,https://www.ef.com.au/


In [17]:
cities_pd.to_csv("Airport_Output.csv", index=False)